In [1]:
%matplotlib auto
import scipy.io as sio
from matplotlib import pyplot as plt
import numpy as np

Using matplotlib backend: Qt5Agg


In [9]:
dI_dV=sio.loadmat('a.mat')['dI_dV']
dI_dV1=sio.loadmat('a1.mat')['dI_dV1']
dI_dV2=sio.loadmat('a2.mat')['dI_dV2']
dI_dV3=sio.loadmat('a3.mat')['dI_dV3']
E=sio.loadmat('a.mat')['E']
E1=sio.loadmat('a1.mat')['E1']
E2=sio.loadmat('a2.mat')['E2']
E3=sio.loadmat('a3.mat')['E3']
E=E[0]
E1=E1[0]
E2=E2[0]
E3=E3[0]

In [10]:
def make_trapazoide_wire(a=1, t=1.0,nt=1.0, L=10, w1=10, w2=30,leads=-1):
    # Start with an empty tight-binding system and a single square lattice.
    # `a` is the lattice constant (by default set to 1 for simplicity).

    lat = kwant.lattice.square(a)

    syst = kwant.Builder()

    #### Define the scattering region. ####
    # Now, we aim for a more complex shape, namely a ring (or annulus)
    def Trapazoide(pos):
        (x, y) = pos
        return ((0 <= x < L) and (0 <= y < (((w2-w1)/L)*x+w1)))

    # and add the corresponding lattice points using the `shape`-function
    syst[lat.shape(Trapazoide, (L-1,1))] = 4 * t
    syst[lat.neighbors()] = -nt*t

    #### Define the leads. ####
    # left lead
    sym_lead = kwant.TranslationalSymmetry((-a, 0))
    lead = kwant.Builder(sym_lead)

    def lead_shape(pos):
        (x, y) = pos
        return (0 <= y < w1)

    lead[lat.shape(lead_shape, (0, 1))] = 4 * t
    lead[lat.neighbors()] = leads

    #### Attach the leads and return the system. ####
    syst.attach_lead(lead)
    syst.attach_lead(lead.reversed())
    return syst

def make_wire_l(a=1, t=1.0, nt=1.0, L=10, W=10,leads=-1,is_periodic=False):

    lat = kwant.lattice.square(a, norbs=1)
    syst = kwant.Builder()

    syst[(lat(i,j) for i in range(L) for j in range(W))] = 4*t
    syst[lat.neighbors()] = -nt*t
    if is_periodic:
        for j in range(W):
            syst[lat(0,j),lat(L-1,j)] = -nt*t
    lead = kwant.Builder(kwant.TranslationalSymmetry([-a,0]))
    lead[(lat(0,j) for j in range(W))] = 4*t
    lead[lat.neighbors()] = leads
    syst.attach_lead(lead)
    lead2 = kwant.Builder(kwant.TranslationalSymmetry([a,0]))
    lead2[(lat(0,j) for j in range(W))] = 4*t
    lead2[lat.neighbors()] = leads
    syst.attach_lead(lead2)
    return syst

def get_normal_eigen(a=1,t=1,nt=1,L=100,W=20,leads=-1,k=50,is_periodic=False):
    syst_finite = make_wire_l(a, t, nt, L, W,leads,is_periodic=is_periodic)
    syst_finite = syst_finite.finalized()
    return calc_eigen(syst_finite,k)

def get_trapazoide_eigen(a=1,t=1,nt=1,L=100,w1=10,w2=30,leads=-1,k=50):
    syst_finite = make_trapazoide_wire(a, t, nt, L, w1,w2,leads)
    syst_finite = syst_finite.finalized()
    return calc_eigen(syst_finite,k)

def calc_eigen(syst_finite,k):
    #evecs = evecs(space,energy)
    ham_mat = syst_finite.hamiltonian_submatrix(sparse=True)
    eigenes,evecs = sla.eigsh(ham_mat, k=k, which='SM')
    length=evecs.shape[1]
    evecs1=np.zeros(evecs.shape,dtype=complex)
    for i in range(length):
        a=np.sqrt(np.sum(np.abs(evecs[:,i])**2))
        evecs1[:,i]=evecs[:,i]/a
    return syst_finite,evecs1,eigenes,ham_mat
    
def get_wire_density_energy(a,t,nt,L,W,leads,k,is_Normal=True,w1=10,w2=30,is_periodic=False,delta_y=0): 
    #abs_evecs = evecs(energy,space)
    #evecs = evecs(space,energy)
    delta_abs=int(W*delta_y/2)
    W_abs=int(W-2*delta_abs)
    total_density_avg=np.zeros(shape=(k,L))
    abs_evecs_avg=np.zeros(shape=(k,L))
    if is_Normal:
        syst_finite,eve,eva,ham= get_normal_eigen(a,t,nt,L,W,leads,k,is_periodic)
        total_density=np.zeros(shape=(L*W,k))
        for i in range(len(eva)):
            abs_evec=(np.abs(eve[:,i])**2)
            total_density[:,i] = np.sum(np.abs(eve[:,:(i+1)])**2,1)
            A=total_density[:,i].reshape(-1,W)
            B=abs_evec.reshape(-1,W);
            total_density_avg[i,:] = np.sum(A[:,delta_abs:(W-delta_abs)],axis=1) #averaged over y
            abs_evecs_avg[i,:] = np.sum(B[:,delta_abs:(W-delta_abs)],axis=1) #averaged over y

    else:
        syst_finite,eve,eva,ham= get_trapazoide_eigen(a,t,nt,L,w1,w2,leads,k)
        total_density=np.zeros(shape=(eve.shape[0],k))
        total_density_avg=np.zeros(shape=(k,L))
        abs_evecs_avg=np.zeros(shape=(k,L))
        total_length=np.zeros(L-1)
        def Trapazoide(pos):
            (x, y) = pos
            return ((0 <= x < L) and (0 <= y < (((w2-w1)/L)*x+w1)))
        lat = kwant.lattice.square(a)
        aa=lat.shape(Trapazoide, (L-1,1))
        AA=list(aa())
#density_per_energy = np.zeros(eve.shape)
        for i in range(len(AA)):
            #Counting the length of each collun
            T=AA[i];
            (x,y)=T[1];
            total_length[x-1]=total_length[x-1]+1
        for i in range(len(eva)):
            abs_evec=(np.abs(eve[:,i])**2)
            total_density[:,i] = np.sum(np.abs(eve[:,:(i+1)])**2,1)
            j=0
            counter=0
            for length in total_length:
                temp =int(j+length)
                total_density_avg[i,counter] = np.sum(total_density[j:temp,i]) #averaged over y
                abs_evecs_avg[i,counter]= np.sum(abs_evec[j:temp])
                j=temp
                counter=counter+1
    return syst_finite,total_density_avg,total_density,eva,abs_evecs_avg,ham,eve

def plot_QPI(X,eva,vmin=-2,vmax=-0.1,delta=0,window=False,a=1):
    L=np.shape(X)[1]
    delta_abs=int(L*delta/2)
    L_abs=int(L-2*delta_abs)
    X_1=X[:,delta_abs:L-delta_abs]
    X_2=np.zeros(X_1.shape)
    real_axis=np.linspace(0,a,L)
    if window:
        for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                X_2[i,j]=X_1[i,j]*np.sin(np.pi*(j*a)/(L_abs))
        DATA=np.log(np.abs(np.fft.fftshift(np.fft.fft(X_2,axis=1),axes=1)))
    else:
        DATA=np.log(np.abs(np.fft.fftshift(np.fft.fft(X_1,axis=1),axes=1)))
    x_axis=np.linspace(0,2*np.pi/a,2*(L_abs))
    x_axis=x_axis-np.mean(x_axis)
    for j in range(len(DATA[:,0])):
        DATA[j,int(DATA.shape[1]/2)]=(DATA[j,int(DATA.shape[1]/2)-1]+DATA[j,int(DATA.shape[1]/2)+1])/2
    fig,ax = plt.subplots(figsize=(10,10))
    cax=ax.imshow(DATA,origin='lower',extent=[np.min(x_axis),np.max(x_axis),np.min(eva),np.max(eva)],aspect='auto',vmin=vmin,vmax=vmax)
    plt.colorbar(cax)
    plt.ylabel('Energy')
    plt.xlabel('q[2*pi/a]')
    plt.title('E vs. q', fontsize=20)
    ax.xaxis.label.set_size(20)
    ax.yaxis.label.set_size(20)
    plt.show()
    
def plot_QPI_bokeh(X,eva,L,vmin=-2,vmax=-0.1):
    DATA=np.log(np.abs(np.fft.fftshift(np.fft.fft(X,axis=1),axes=1)))
    x_axis=np.linspace(0,2*np.pi/a,2*L);
    x_axis=x_axis-np.mean(x_axis);
    for j in range(len(DATA[:,0])):
        DATA[j,int(L/2)]=(DATA[j,int(L/2)-2]+DATA[j,int(L/2)+1])/2
    p=figure(x_range=(min(x_axis),max(x_axis)),y_range=(min(eva),max(eva)))
    color_mapper = LinearColorMapper(palette="Viridis256")
    p.image(image=[DATA],x=x_axis,y=eva,dw=2*L,dh=max(eva),color_mapper=color_mapper)
    color_bar = ColorBar(color_mapper=color_mapper,
                     label_standoff=12, border_line_color=None, location=(0,0))
    p.add_layout(color_bar, 'left')
    show(p)
    
def plot_spectra(X,eva,vmin=0,vmax=3):
    L=np.shape(X)[1]
    plt.figure(figsize=(10,10))
    plt.ylabel('Energy', fontsize=20)
    plt.xlabel('x[a]', fontsize=20)
    plt.title('E vs. x (y averaged)', fontsize=20)
    plt.imshow(X,origin='lower',aspect='auto',extent=[0,L,np.min(eva),np.max(eva)],vmin=vmin,vmax=vmax);
    plt.colorbar()
    
def plot_di_dv(X,eva,ax=None,title=None,xlabel=None,ylabel=None):
    L=np.shape(X)[1]
    if ylabel is None:
        ylabel = 'dI/dV'
    if xlabel is None:
        xlabel = 'dI/dV'
    if title is None:
        title ='Average dI/dV'
    if ax is None:
        plt.figure(figsize=(10,10))
        Y=(np.mean(X,1))
        Ya=np.diff(Y)
        Yc=np.diff(eva)
        Yc1=Ya/Yc
        plt.plot(eva,(np.mean(X,1)),'.') #averaging X
        plt.ylabel(ylabel, fontsize=20)
        plt.xlabel(xlabel, fontsize=20)
        plt.title(title, fontsize=20)
    else:
        ax.plot(eva,(np.mean(X,1)),'.') #averaging X
        ax.set_ylabel(ylabel, fontsize=20)
        ax.set_xlabel(xlabel, fontsize=20)
        ax.set_title(title, fontsize=20)
        
def sum_N_consequtive(x,n):
    return np.array([sum(x[i:i+n]) for i in range(0, len(x), n)])

def Average_dI_dV(X,eva,binning=1,overlap=0):
#params : 
# X(E,x) is a matrix that has in its i,j place ||Psi(E,X)||^2
# eva is the eigen vectors of the problem
# returns
# X(E_avg,x),E_avg the averaged di_dv according to a moving average with overlap=overlap and binning of binning.
    eva_len=len(eva)
    x_len=np.shape(X)[1]
    sorting_idx=np.argsort(eva)
    sorted_eva=eva[sorting_idx]
    X_sorted=X[(sorting_idx),:]
    movement=int(binning-overlap)
    energy_count_size=int(np.ceil(float(eva_len-binning)/float(movement))+1)
    energy_count_size_orig=np.ceil(float(eva_len)/float(binning))
    X_binned=np.zeros((energy_count_size,x_len));
    avg_energies=np.zeros(energy_count_size);
    first_energy=sorted_eva[0];
    second_energy=0;
    j=binning-1;
    i=0;
    while j<=eva_len-1:
        first_idx=j-binning+1;
        first_energy=sorted_eva[first_idx];
        second_energy=sorted_eva[j];
        avg_energies[i]=np.mean(sorted_eva[first_idx:j+1]);
        X_binned[i,:]=np.sum(X_sorted[first_idx:j+1,:],0);
        j=j+movement;
        i=i+1;
    if j<(eva_len+movement-1):
        num=binning-(j-eva_len+1);
        first_idx=j-num-1;
        first_energy=sorted_eva[first_idx];
        second_energy=sorted_eva[-1];
        avg_energies[i]=np.mean(sorted_eva[first_idx:]);
        X_binned[i,:]=np.sum(X_sorted[first_idx:,:],0);
    dE=np.diff(avg_energies);
    dX_binned=np.diff(X_binned,axis=0);
    dNx_dE=dX_binned/dE[:,None] #Weird syntex;
    dNx_dE=dNx_dE*float(energy_count_size_orig/energy_count_size);
    avg_E_final=np.mean([avg_energies[:-1],avg_energies[1:]],axis=0);
    print(np.sum(X_binned),np.sum(X),np.sum(X_sorted),float(energy_count_size_orig/energy_count_size))
    return dNx_dE, avg_E_final;

def Average_dI_dV_T(X,eva,binning=1,overlap=0):
#params : 
# X(E,x) is a matrix that has in its i,j place ||Psi(E,X)||^2
# eva is the eigen vectors of the problem
# returns
# X(E_avg,x),E_avg the averaged di_dv according to a moving average with overlap=overlap and binning of binning.
    eva_len=len(eva)
    x_len=np.shape(X)[1]
    sorting_idx=np.argsort(eva)
    sorted_eva=eva[sorting_idx]
    X_sorted=X[(sorting_idx),:]
    movement=int(binning-overlap)
    energy_count_size=int(np.ceil(float(eva_len-binning)/float(movement))+1)
    energy_count_size_orig=np.ceil(float(eva_len)/float(binning))
    X_binned=np.zeros((energy_count_size,x_len));
    avg_energies=np.zeros(energy_count_size);
    dE=np.zeros(energy_count_size);
    first_energy=sorted_eva[0];
    second_energy=0;
    j=binning-1;
    i=0;
    if binning==1:
        avg_energies=sorted_eva;
        X_binned=X_sorted;
        dE=np.diff(avg_energies);
        dE=np.append(dE,dE[-1])
    else:
        while j<=eva_len-1:
            first_idx=j-binning+1;
            first_energy=sorted_eva[first_idx];
            second_energy=sorted_eva[j];
            dE[i]=second_energy-first_energy;
            avg_energies[i]=np.mean(sorted_eva[first_idx:j+1]);
            X_binned[i,:]=np.sum(X_sorted[first_idx:j+1,:],0);
            j=j+movement;
            i=i+1;
        if j<(eva_len+movement-1):
            num=binning-(j-eva_len+1);
            first_idx=j-binning+1;
            first_energy=sorted_eva[first_idx];
            second_energy=sorted_eva[-1];
            dE[i]=second_energy-first_energy;
            avg_energies[i]=np.mean(sorted_eva[first_idx:]);
            X_binned[i,:]=np.sum(X_sorted[first_idx:,:],0);
    dNx_dE=X_binned/dE[:,None] #Weird syntex;
    dNx_dE=dNx_dE*float(energy_count_size_orig/energy_count_size);
    print(np.sum(X_binned),np.sum(X),np.sum(X_sorted),float(energy_count_size_orig/energy_count_size),np.min(dE))
    return X_binned,dE,dNx_dE, avg_energies

         
class Formatter(object):
    def __init__(self, im):
        self.im = im
    def __call__(self, x, y):
        z = self.im.get_array()[int(y), int(x)]
        return 'x={:.01f}, y={:.01f}, z={:.01f}'.format(x, y, z)

In [13]:
#plot_spectra(dI_dV,E,vmin=0,vmax=0.009)
#plot_spectra(dI_dV1,E1,vmin=0,vmax=0.009)
#plot_spectra(dI_dV2,E2,vmin=0,vmax=0.009)
plot_spectra(dI_dV3,E3,vmin=0,vmax=0.009)

In [19]:
dI_dV=sio.loadmat('a.mat')['dI_dV']
avg_energies=sio.loadmat('a.mat')['avg_energies']
#dI_dV=sio.loadmat('b.mat')['dI_dV']
#avg_energies=sio.loadmat('b.mat')['avg_energies']

In [20]:
plot_spectra(dI_dV,avg_energies,vmin=0,vmax=0.027)


In [17]:
plot_QPI(dI_dV,avg_energies,vmin=-7,vmax=0.027)

In [21]:
np.size(avg_energies)

424